In [1]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
tf.enable_eager_execution()


In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 30 #@param {type:"integer"}

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [9]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
import os
from os import listdir
from os.path import join
 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.preprocessing.image import img_to_array, load_img
 
 
 
 
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
 
 
def save_tf_records(x_train, y_train, out_path):
    writer = tf.python_io.TFRecordWriter(out_path)
 
    for i in range(y_train.shape[0]):
   
        example = tf.train.Example(features=tf.train.Features(
            feature={'image': _bytes_feature(x_train[i].tostring()),
                     'labels': _bytes_feature(
                         y_train[i].tostring())
                     }))
 
        writer.write(example.SerializeToString())
 
    writer.close()

In [0]:
save_tf_records(x_train, y_train, './training.tf_records')

In [0]:
def load_tf_records(path):
    dataset = tf.data.TFRecordDataset(path)
 
    def parser(record):
        featdef = {
            'image': tf.FixedLenFeature(shape=[], dtype=tf.string),
            'labels': tf.FixedLenFeature(shape=[], dtype=tf.string),
        }
 
        example = tf.parse_single_example(record, featdef)
        im = tf.decode_raw(example['image'], tf.float32)
        im = tf.reshape(im, (-1, 40, 40, 3))
        lbl = tf.decode_raw(example['labels'], tf.int64)
        return im, lbl
 
    dataset = dataset.map(parser)
    #dataset = dataset.shuffle(buffer_size=50000)
    dataset = dataset.batch(50000)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [0]:
x_train,y_train = load_tf_records('./training.tf_records')


In [0]:
x_train.shape

TensorShape([Dimension(50000), Dimension(1), Dimension(40), Dimension(40), Dimension(3)])

In [0]:
##Implementing CutOut
def get_random_eraser(input_img,p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    #cutout
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img
    
    return eraser(input_img)


def tf_Cutout_image(image , label):
  imshape = image.shape
  [image,] = tf.py_function(get_random_eraser,[image],[tf.float32])
  image.set_shape(imshape)
  return image 

  

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1
last_batch_size = len_train % BATCH_SIZE

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf_Cutout_image(tf.random_crop(x, [32, 32, 3]),y)), y)

In [0]:
#data_aug = lambda x, y: (tf.image.random_flip_left_right(x, y))

In [0]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        print(input_img.shape[1:])
        img_h, img_w, img_c = input_img.shape[1:]
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
HEIGHT = 32
WIDTH = 32
NUM_CHANNELS = 3
#eraser = get_random_eraser()
def augmentation(x, y):
    # x = tf.image.resize_with_crop_or_pad(
    #     x, HEIGHT + 8, WIDTH + 8)
    # x = tf.image.random_crop(x, [-1,HEIGHT, WIDTH, NUM_CHANNELS])
    # x = tf.image.random_brightness(x, max_delta=0.5)
    # x = tf.image.random_contrast(x, lower=0.5,upper=1.5)
    #x = tf_Cutout_image(x , y)
    #x = eraser(x)
    x = tf.image.random_flip_left_right(x)
    return x, y

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(35):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(augmentation).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
  #train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1) 
  
  #Since you are using dataset.prefetch(buffer_size=1) after dataset.batch(), it means that it will prefetch 1 batch.

  tf.keras.backend.set_learning_phase(1)
  batch_count = 0
  for (x, y) in tqdm(train_set):
    batch_count += 1
    if batch_count <batches_per_epoch :
        x = tf.reshape(x, [BATCH_SIZE,40,40,3])
        y = tf.reshape(y, [BATCH_SIZE])
    else:
        x = tf.reshape(x, [last_batch_size,40,40,3])
        y = tf.reshape(y, [last_batch_size])
        
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

epoch: 1 lr: 0.08 train loss: 2.323457322998047 train acc: 0.10448 val loss: 2.3219630310058594 val acc: 0.1081 time: 66.21863079071045


epoch: 2 lr: 0.16 train loss: 2.323396055908203 train acc: 0.1049 val loss: 2.321799841308594 val acc: 0.1085 time: 131.4104676246643


epoch: 3 lr: 0.24 train loss: 2.323381613769531 train acc: 0.10526 val loss: 2.321932666015625 val acc: 0.1078 time: 196.18572354316711


epoch: 4 lr: 0.32 train loss: 2.3233933337402344 train acc: 0.10508 val loss: 2.3220723999023436 val acc: 0.1082 time: 261.0731554031372


epoch: 5 lr: 0.4 train loss: 2.323539405517578 train acc: 0.10458 val loss: 2.3218971435546876 val acc: 0.1082 time: 326.0828003883362


...

epoch: 15 lr: 0.18947368421052635 train loss: 2.323333083496094 train acc: 0.10526 val loss: 2.321892687988281 val acc: 0.1078 time: 971.610987663269


epoch: 16 lr: 0.16842105263157897 train loss: 2.3231998413085937 train acc: 0.1051 val loss: 2.3218245239257813 val acc: 0.108 time: 1036.0229194164276


epoch: 17 lr: 0.1473684210526316 train loss: 2.323219030761719 train acc: 0.10498 val loss: 2.3219602111816404 val acc: 0.1076 time: 1100.9592702388763


epoch: 18 lr: 0.12631578947368421 train loss: 2.3232056213378907 train acc: 0.10596 val loss: 2.321886865234375 val acc: 0.1081 time: 1165.3227517604828


epoch: 19 lr: 0.10526315789473689 train loss: 2.3234270593261717 train acc: 0.10504 val loss: 2.3220576416015626 val acc: 0.1082 time: 1229.8350365161896


In [0]:
# t = time.time()
# test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

# for epoch in range(2):
#   train_loss = test_loss = train_acc = test_acc = 0.0
#   train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
#   #train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(augmentation).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

#   tf.keras.backend.set_learning_phase(1)
#   for (x, y) in tqdm(train_set):
#     with tf.GradientTape() as tape:
#       loss, correct = model(x, y)

#     var = model.trainable_variables
#     grads = tape.gradient(loss, var)
#     for g, v in zip(grads, var):
#       g += v * WEIGHT_DECAY * BATCH_SIZE
#     opt.apply_gradients(zip(grads, var), global_step=global_step)

#     train_loss += loss.numpy()
#     train_acc += correct.numpy()

#   tf.keras.backend.set_learning_phase(0)
#   for (x, y) in test_set:
#     loss, correct = model(x, y)
#     test_loss += loss.numpy()
#     test_acc += correct.numpy()
    
#   print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

ValueError: ignored